# Rough Emission

This worked example shows how to use `roughness` to compute a radiated emission spectrum from a rough surface.

## Setup

Make sure to install the `roughness` package and download the default lookup tables before running this example. See the [getting started](../getting_started/) page for more information.

## Imports

The `roughness` package provides a `RoughnessThermalModel` class that can be used to compute emission spectra from rough surfaces. The `xarray` package is used to store and manipulate the lookup tables.

In [ ]:
from roughness import RoughnessThermalModel

# Create a model instance
rtm = RoughnessThermalModel()
rtm

## Custom Lookup Tables

The default lookup tables have been developed and tested for the Moon. However, `roughness` allows users to generate custom lookup tables for any body with a custom thermal model.

The `shadow_lookup` table is a lookup table of probabilities that a given surface facet is in shadow. It is generated using a raytracing algorithm applied to a self-affine fractal rough surface with a default Hurst exponent chosen for lunar applications. The default surface should function for most rough planetary surfaces, but can be regenerated ysing the `roughness.make_los_table` module as described in [shadow_tables](../shadow_tables/).

The `temperature_lookup` table is a lookup table of surface temperatures for each surface facet. Any thermal model can be used to generate this table and it can have arbitrary dimensionality, the only requirement is that the output is an `xarray.Dataset` with a data variable `tsurf` and contains at least the `theta` and `az` coordinates, corresponding to surface facet slope and azimuth. 

The default temperature table used in `roughness` was developed for the Moon using KRC ([krc.mars.asu.edu](https://krc.mars.asu.edu)) and **is** body-specific due to the unique thermal environment of each planetary body. Below is the temperature Dataset with coordinates relevant to predicting surface temperatures on the Moon. See [temperature_tables](../temperature_tables/) for more details.

In [ ]:
rtm.temperature_ds

The `roughness` thermal model interpolates between the pre-computed shadowing and lookup tables to compute the emitted radiance for a given geometry and local conditions. The desired parameters are passed to the `emission` method of the `RoughnessThermalModel` object, which returns an `xarray` object containing the emitted radiance for each wavelength and roughness value. The `emission` method requires the following parameters:

- `geometry`: A tuple containing the solar incidence angle, solar azimuth, spacecraft emission angle, and spacecraft azimuth, all in units of degrees. Azimuths are computed clockwise from North.
- `wavelengths`: An array of wavelengths in units of microns where emission will be computed.
- `rms`: The RMS roughness in units of degrees.
- `tparams`: A dictionary containing the parameters required to interpolate the temperature lookup table. The required parameters are all coordinates of `rtm.temperature_ds` except for facet `theta` and `az`. Print `rtm.required_tparams` to check which parameters are needed.
- `kwargs`: Additional keyword arguments are passed to `roughness.emission.rough_emission_lookup` to fine tune the emission run. See [emission](../reference/emission/) for more details.

In [ ]:
rtm.required_tparams

## Predicting emission

We can now use the roughness model to predict the emission from a surface with a given roughness. We'll use an RMS value of 25 degrees and pick a point near the equator. We use a helper function to compute the local time at the chosen point.

In [ ]:
import numpy as np
from roughness import helpers as rh

# Set parameters
rms = 15
wls = np.linspace(0.45, 100, 1000)
sun_theta, sun_az = 30, 90
sc_theta, sc_az = 60, 90
lat = 0  # Latitude [degrees]
ls = 0  # Solar longitude (L_s) [degrees]
albedo = 0.12  # Directional hemispherical (thermal) albedo

# Compute tloc and set tparams
tloc = rh.inc_to_tloc(sun_theta, sun_az, lat)
tparams = {'albedo': albedo, 'lat': lat, 'ls': ls, 'tloc': tloc}

# Compute rough emission
emission = rtm.emission(sun_theta, sun_az, sc_theta, sc_az, wls, rms, tparams)

emission.plot()

## Plotting brightness tempertures



In [ ]:
from roughness import emission as re

# Convert emitted radiance to brightness temperature
btemp = re.btempw(wls, re.mrad2cmrad(emission))
btemp.name = "Brightness Temperature [K]"

# Plot
btemp.plot()